
# Vardef demo

Denne demonstrasjonen viser hvordan du kan bruke Vardef-modulen i Dapla toolbelt metadata for å opprette, forvalte og hente ut variabeldefinisjoner. 

Vi anbefaler å bruke Vardef-modeller for å opprette og redigere variabeldefinisjoner.

Nyttige modeller:

- Draft
- Owner
- Contact
- UpdateDraft
- Patch
- ValidityPeriod

Les her om  [alternativer til Model](#alternativer-til-model-input)

In [1]:
from datetime import date
from pprint import pprint

from dapla_metadata.variable_definitions.generated.vardef_client.models.contact import (
    Contact,
)
from dapla_metadata.variable_definitions.generated.vardef_client.models.draft import (
    Draft,
)
from dapla_metadata.variable_definitions.generated.vardef_client.models.update_draft import (
    UpdateDraft,
)
from dapla_metadata.variable_definitions.generated.vardef_client.models.validity_period import (
    ValidityPeriod,
)
from dapla_metadata.variable_definitions.generated.vardef_client.models.variable_status import (
    VariableStatus,
)
from dapla_metadata.variable_definitions.vardef import Vardef



## Drafts

### Opprette et utkast

Alle variabeldefinisjoner får status `DRAFT` ved opprettelse slik at innholdet kan kvalitetsikres før man publiserer.

Det er kun i `DRAFT` at man kan endre `short_name`og det er kun i `DRAFT` at man kan slette en lagret variabeldefinisjon. 

Kan gi en `409 CONFLICT` hvis short_name allerede eksisterer, da kortnavnet må være unikt.

Den som oppretter variabeldefinisjonen blir lagret som `Owner` (teamet og gruppen man representerer).

Alle operasjoner for lagrede variabeldefinisjoner må utføres på en variabel.


In [2]:
draft = Draft(
    name = {
        "nb": "test navn",
        "nn": "test namn",
        "en": "test name",
    },
    short_name= "test_shortname",
    definition= {
        "nb": "Skriv en definisjonstekst på norsk bokmål.",
        "nn": "Skriv ein definisjonstekst på nynorsk.",
        "en": "Write a definition text in english.",
    },
    classification_reference="91",
    unit_types=[
        "01",
    ],
    subject_fields=[
        "al",
    ],
    contains_special_categories_of_personal_data=False,
    measurement_type=None,
    valid_from=date(2024,12,18),
    external_reference_uri=None,
    comment=None,
    related_variable_definition_uris=None,
    contact=None,
)

# Assign value VariableDefinition to variable.
my_draft = Vardef.create_draft(draft)




### Eksempel `exception` ved opprettelse av utkast

Vardef-modulen er designet for å hjelpe brukeren med å forvalte variabeldefinisjoner og kvalitetsikre at alle variabeldefinisjoner oppfyller nødvendige krav og regler.

I mange tilfeller blir `VardefClientException` returnert til brukeren med en tydelig feilmelding. Meldingen vil komme på bunnen av `tracestack`.

Det er nyttig å gjøre seg kjent med de ulike feilmeldingene da de gir viktig informasjon om hva som har gått galt og hvor feilen ligger. 

I eksempelet er det ugyldige Klass-koder som utløser en `BadRequestException` (400).


In [3]:
invalid_draft = Draft(
    name = {
        "nb": "test navn",
        "nn": "test namn",
        "en": "test name",
    },
    short_name= "test_excpetion",
    definition= {
        "nb": "Skriv en definisjonstekst på norsk bokmål.",
        "nn": "Skriv ein definosjonstekst på nynorsk.",
        "en": "Write a definition text in english.",
    },
    classification_reference="91",
    unit_types=[
        "haha",
    ],
    subject_fields=[
        "a",
    ],
    contains_special_categories_of_personal_data=False,
    measurement_type=None,
    valid_from=date(2024,12,18),
    external_reference_uri=None,
    comment=None,
    related_variable_definition_uris=None,
    contact=None,
)

Vardef.create_draft(invalid_draft)

VardefClientException: Status 400: 
createVariableDefinition.draft.unitTypes[0]<list element>: Code haha is not a member of classification with id 702
createVariableDefinition.draft.subjectFields[0]<list element>: Code a is not a member of classification with id 618


#### Nyttig informasjon om variabler

Hvis du trenger å tilordne variabeldefinisjonen din til en variabel igjen og har tilgang til variabeldefinisjon-id:

> my_new_variable = Vardef.get_variable_definition("kWzf65gt") 


Hvis du har mistet oversikten over variabeldefinisjons-id, finnes det eksempler på hvordan du kan hente en spesifikk variabeldefinisjon [her](#hente-en-variabeldefinisjon-ved-hjelp-av-kortnavn)


In [4]:
# Vi kan få tilgang til alle feltene direkte fra variabelen.
print("Definisjonstekst på bokmål: ")
pprint(my_draft.definition["nb"])

# Eller du kan få tilgang til feltene ved å hente den lagrede variabeldefinisjonen etter ID for å sjekke verdiene.
print("\nVariabeldefinisjon opprettet: ")
print(Vardef.get_variable_definition(my_draft.id).created_at)

# Feltene blir ikke automatisk oppdatert etter endringer. Hvis du ønsker å fortsette å få tilgang til feltene direkte fra variabelen, kan du oppdatere på denne måten etter hver endring:
my_draft = Vardef.get_variable_definition(my_draft.id)



Definisjonstekst på bokmål: 
'Skriv en definisjonstekst på norsk bokmål.'

Variabeldefinisjon opprettet: 
2024-12-20 15:15:29.576000


### Rediger et utkast

Denne operasjonen kan kun utføres på variabeldefinisjoner med status DRAFT. Alle operasjoner på lagrede variabeldefinisjoner er kun tilgjengelige for `Owner`.


In [5]:
# Oppdater kontaktinformasjon
my_contact = Contact(
    title={
        "nb": "Seksjon for befolkningsstatistikk",
        "nn": "Seksjon for befolkningsstatistikk",
        "en": "Division for population statistics",
    },
    email="sibby@ssb.no",
)
update_contact = UpdateDraft(
    contact=my_contact,
)

my_draft.update_draft(update_draft=update_contact)
my_draft = Vardef.get_variable_definition(my_draft.id)

print(my_draft.contact)

{'title': {'nb': 'Seksjon for befolkningsstatistikk', 'nn': 'Seksjon for befolkningsstatistikk', 'en': 'Division for population statistics'}, 'email': 'sibby@ssb.no'}


### Eksempel `ValidationError` ved oppdatering av utkast

Ugyldig kortnavn vil returnerer en `ValidationError`.

In [6]:
invalid_short_name = UpdateDraft(
    short_name="_)(45)",
)
my_draft.update_draft(invalid_short_name)


ValidationError: 1 validation error for UpdateDraft
short_name
  Value error, must validate the regular expression /^[a-z0-9_]{3,}$/ [type=value_error, input_value='_)(45)', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error

### Migrere en variabeldefinisjon fra Vardok til Vardef

Etter migreringen blir variabelen lagret som `DRAFT` og kan redigeres på samme måte som vist i [Rediger et utkast](#rediger-et-utkast)




In [7]:
my_vardok_draft = Vardef.migrate_from_vardok("90")
print(my_vardok_draft)

{
  "id": "A1i3HFZe",
  "patch_id": 1,
  "name": {
    "nb": "Spesifisert registreringstype",
    "nn": null,
    "en": null
  },
  "short_name": "spes_reg_type",
  "definition": {
    "nb": "Variabelen viser kode for personens særforhold i folkeregistreringen",
    "nn": null,
    "en": null
  },
  "classification_reference": null,
  "unit_types": [
    "20"
  ],
  "subject_fields": [],
  "contains_special_categories_of_personal_data": false,
  "variable_status": "DRAFT",
  "measurement_type": null,
  "valid_from": "1996-06-01",
  "valid_until": null,
  "external_reference_uri": "https://www.ssb.no/a/xml/metadata/conceptvariable/vardok/90",
  "comment": null,
  "related_variable_definition_uris": [],
  "owner": {
    "team": "dapla-felles",
    "groups": [
      "dapla-felles-developers"
    ]
  },
  "contact": null,
  "created_at": "2024-12-20T15:18:17.537909",
  "created_by": null,
  "last_updated_at": "2024-12-20T15:18:17.537909",
  "last_updated_by": null
}


#### Eksempel på `exception` ved migrering 

In [8]:
# Id eksisterer ikke
Vardef.migrate_from_vardok("21")

VardefClientException: Status 404: Vardok id 21 not found

### Slette et utkast

Denne handlingen kan kun utføres på en variabeldefinisjon med status `DRAFT`.

In [9]:
my_vardok_draft.delete_draft()

'Variable A1i3HFZe safely deleted'

## Publisere

Når utkastet har blitt kvalitetssjekket er det på tide å publisere. 

Vi har disse statusene:

- DRAFT
- PUBLISHED_INTERNAL
- PUBLISHED_EXTERNAL

Vi anbefaler å benytte seg av variabelstatus-modellen for å motvirke skrivefeil og lignende.

For eksempel: 

> VariableStatus.PUBLISHED_INTERNAL 

Merk at denne prosessen er irreversibel. 

Det er ulike regler som gjelder ved publisering internt og eksternt, men merk også her at når en variabeldefinisjon er publisert eksternt kan den ikke endres tilbake til å kun være publisert internt.



In [10]:
update_status = UpdateDraft(
    variable_status=VariableStatus.PUBLISHED_INTERNAL,
)

my_draft.update_draft(update_status)

CompletePatchOutput(id='QlTc_6H5', patch_id=1, name={'nb': 'test navn', 'nn': 'test namn', 'en': 'test name'}, short_name='test_shortname', definition={'nb': 'Skriv en definisjonstekst på norsk bokmål.', 'nn': 'Skriv ein definisjonstekst på nynorsk.', 'en': 'Write a definition text in english.'}, classification_reference='91', unit_types=['01'], subject_fields=['al'], contains_special_categories_of_personal_data=False, variable_status=<VariableStatus.PUBLISHED_INTERNAL: 'PUBLISHED_INTERNAL'>, measurement_type=None, valid_from=datetime.date(2024, 12, 18), valid_until=None, external_reference_uri=None, comment=None, related_variable_definition_uris=None, owner={'team': 'dapla-felles', 'groups': ['dapla-felles-developers']}, contact={'title': {'nb': 'Seksjon for befolkningsstatistikk', 'nn': 'Seksjon for befolkningsstatistikk', 'en': 'Division for population statistics'}, 'email': 'sibby@ssb.no'}, created_at=datetime.datetime(2024, 12, 20, 15, 15, 29, 576000), created_by=None, last_update

## Patch

### Opprette en Patch

Patcher brukes for mindre endringer som ikke krever en ny `ValidityPeriod`.

Eksempler på grunner for å opprette en ny patch:

- Korrektur av en skrivefeil
- Legge til en oversettelse
- Legge til `subject_field`

Oppgi kun de feltene som skal endres. Andre felter vil beholde sine nåværende verdier.

#### Eksempel oppdatere `Owner`

Legge til gruppe som kan forvalte variabeldefinisjonen. 

In [11]:
from dapla_metadata.variable_definitions.generated.vardef_client.models.owner import (
    Owner,
)
from dapla_metadata.variable_definitions.generated.vardef_client.models.patch import (
    Patch,
)

new_owner = Patch(
    owner=Owner(
    team="dapla-felles",
    groups=[
        "dapla-felles-developers",
        "play-enhjoern-a-developers",
    ],
),
)

my_draft.create_patch(new_owner)

my_draft = Vardef.get_variable_definition(my_draft.id)

print(my_draft.owner)

{'team': 'dapla-felles', 'groups': ['dapla-felles-developers', 'play-enhjoern-a-developers']}


#### Overføre eierskap til et annet team

Ved for eksempel omorganiseringer kan det være nødvendig å overføre hele eierskapet til et annet team.

Merk: Når teamet du representerer blir helt fjernet som eier, vil du ikke ha tilgang til å vedlikeholde variabeldefinisjonen.



In [12]:
# Opprette en nytt utkast for å demonstrere overføring av eierskap
owner_draft = Draft(
    name = {
        "nb": "test navn",
        "nn": "test namn",
        "en": "test name",
    },
    short_name= "test_owner_2",
    definition= {
        "nb": "Skriv en definisjonstekst på norsk bokmål.",
        "nn": "Skriv ein definisjonstekst på nynorsk.",
        "en": "Write a definition text in english.",
    },
    classification_reference="91",
    unit_types=[
        "01",
    ],
    subject_fields=[
        "al",
    ],
    contains_special_categories_of_personal_data=False,
    measurement_type=None,
    valid_from=date(2024,12,18),
    external_reference_uri=None,
    comment=None,
    related_variable_definition_uris=None,
    contact=None,
)

transfer_owner_draft = Vardef.create_draft(owner_draft)

transfer_owner_draft.update_draft(UpdateDraft(variable_status=VariableStatus.PUBLISHED_INTERNAL))

replace_owner = Patch(
    owner=Owner(
    team="play-enhjoern-a",
    groups=[
        "play-enhjoern-a-developers",
    ],
),
)

# Oppdatere `Owner``
transfer_owner_draft.create_patch(replace_owner)

CompletePatchOutput(id='7nds8Dqf', patch_id=2, name={'nb': 'test navn', 'nn': 'test namn', 'en': 'test name'}, short_name='test_owner_2', definition={'nb': 'Skriv en definisjonstekst på norsk bokmål.', 'nn': 'Skriv ein definisjonstekst på nynorsk.', 'en': 'Write a definition text in english.'}, classification_reference='91', unit_types=['01'], subject_fields=['al'], contains_special_categories_of_personal_data=False, variable_status=<VariableStatus.PUBLISHED_INTERNAL: 'PUBLISHED_INTERNAL'>, measurement_type=None, valid_from=datetime.date(2024, 12, 18), valid_until=None, external_reference_uri=None, comment=None, related_variable_definition_uris=None, owner={'team': 'play-enhjoern-a', 'groups': ['play-enhjoern-a-developers']}, contact=None, created_at=datetime.datetime(2024, 12, 20, 15, 20, 33, 228770), created_by=None, last_updated_at=datetime.datetime(2024, 12, 20, 15, 20, 33, 228770), last_updated_by=None)

#####  Forsøk på å opprette en ny patch etter overføring av eierskap

Vil føre til en `ForbiddenException`(403)

In [13]:
transfer_owner_draft.create_patch(Patch(unit_types=["01"]))

VardefClientException: Status 403: Forbidden

### Liste patcher for en variabeldefinisjon

In [14]:
patches = my_draft.list_patches()

print(f"Patcher for id {my_draft.id}:\n")
for patch in patches:
    print(f"Patch id: {patch.patch_id}\nOpprettet: {patch.created_at}\n")



Patcher for id QlTc_6H5:

Patch id: 1
Opprettet: 2024-12-20 15:15:29.576000

Patch id: 2
Opprettet: 2024-12-20 15:20:17.656000



### Hent en patch

In [15]:
# Bruk patch_id
print(my_draft.get_patch(2))

{
  "id": "QlTc_6H5",
  "patch_id": 2,
  "name": {
    "nb": "test navn",
    "nn": "test namn",
    "en": "test name"
  },
  "short_name": "test_shortname",
  "definition": {
    "nb": "Skriv en definisjonstekst på norsk bokmål.",
    "nn": "Skriv ein definisjonstekst på nynorsk.",
    "en": "Write a definition text in english."
  },
  "classification_reference": "91",
  "unit_types": [
    "01"
  ],
  "subject_fields": [
    "al"
  ],
  "contains_special_categories_of_personal_data": false,
  "variable_status": "PUBLISHED_INTERNAL",
  "measurement_type": null,
  "valid_from": "2024-12-18",
  "valid_until": null,
  "external_reference_uri": null,
  "comment": null,
  "related_variable_definition_uris": null,
  "owner": {
    "team": "dapla-felles",
    "groups": [
      "dapla-felles-developers",
      "play-enhjoern-a-developers"
    ]
  },
  "contact": {
    "title": {
      "nb": "Seksjon for befolkningsstatistikk",
      "nn": "Seksjon for befolkningsstatistikk",
      "en": "Divi

## Validity period

For å opprette en ny gyldighetsperiode må inndataene inneholde oppdatert `definition`-tekst for alle gjeldende språk og en ny gyldig fra dato.

En ny gyldighetsperiode bør kun opprettes når den grunnleggende definisjonen av variabelen har endret seg.

På denne måten kan den forrige definisjonen bevares for bruk i historiske data.


In [16]:

valid_validity_period = ValidityPeriod(
    definition={
        "nb": "ny definisjon2",
        "nn": "ny definisjon2",
        "en": "new definition2",
    },
    valid_from=date(2040,4,5),
)
my_draft.create_validity_period(valid_validity_period)



CompletePatchOutput(id='QlTc_6H5', patch_id=4, name={'nb': 'test navn', 'nn': 'test namn', 'en': 'test name'}, short_name='test_shortname', definition={'nb': 'ny definisjon2', 'nn': 'ny definisjon2', 'en': 'new definition2'}, classification_reference='91', unit_types=['01'], subject_fields=['al'], contains_special_categories_of_personal_data=False, variable_status=<VariableStatus.PUBLISHED_INTERNAL: 'PUBLISHED_INTERNAL'>, measurement_type=None, valid_from=datetime.date(2040, 4, 5), valid_until=None, external_reference_uri=None, comment=None, related_variable_definition_uris=None, owner={'team': 'dapla-felles', 'groups': ['dapla-felles-developers', 'play-enhjoern-a-developers']}, contact={'title': {'nb': 'Seksjon for befolkningsstatistikk', 'nn': 'Seksjon for befolkningsstatistikk', 'en': 'Division for population statistics'}, 'email': 'sibby@ssb.no'}, created_at=datetime.datetime(2024, 12, 20, 15, 20, 57, 647870), created_by=None, last_updated_at=datetime.datetime(2024, 12, 20, 15, 20,

#### Eksempel ugyldig `ValidityPeriod`

In [17]:
invalid_validity_period = ValidityPeriod(
    name={
        "nb": "nytt navn",
    },
    valid_from=date(2025,4,5),
)

my_draft.create_validity_period(invalid_validity_period)

ValidationError: 1 validation error for ValidityPeriod
definition
  Field required [type=missing, input_value={'name': {'nb': 'nytt nav...tetime.date(2025, 4, 5)}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing

### Liste gyldighetsperioder for en variabeldefinisjon

In [18]:
validity_periods = my_draft.list_validity_periods()

print(f"Gyldighetsperioder for id {my_draft.id}:\n")
for period in validity_periods:
    print(f"Patch id: {period.patch_id}\nGyldig fra: {period.valid_from} til {period.valid_until}\nDefinisjon: {period.definition}\n")



Gyldighetsperioder for id QlTc_6H5:

Patch id: 3
Gyldig fra: 2024-12-18 til 2040-04-04
Definisjon: {'nb': 'Skriv en definisjonstekst på norsk bokmål.', 'nn': 'Skriv ein definisjonstekst på nynorsk.', 'en': 'Write a definition text in english.'}

Patch id: 4
Gyldig fra: 2040-04-05 til None
Definisjon: {'nb': 'ny definisjon2', 'nn': 'ny definisjon2', 'en': 'new definition2'}



## Liste alle variabeldefinisjoner

Nyeste versjon av alle variabeldefinisjoner hentes ut.

In [19]:
variable_definitions = Vardef.list_variable_definitions()
for variable in variable_definitions:
    print(f"Id: {variable.id}\nNavn: {variable.name}\nKortnavn: {variable.short_name}\nDefinisjon: {variable.definition}\nEier: {variable.owner}\n")



Id: 7nds8Dqf
Navn: {'nb': 'test navn', 'nn': 'test namn', 'en': 'test name'}
Kortnavn: test_owner_2
Definisjon: {'nb': 'Skriv en definisjonstekst på norsk bokmål.', 'nn': 'Skriv ein definisjonstekst på nynorsk.', 'en': 'Write a definition text in english.'}
Eier: {'team': 'play-enhjoern-a', 'groups': ['play-enhjoern-a-developers']}

Id: QlTc_6H5
Navn: {'nb': 'test navn', 'nn': 'test namn', 'en': 'test name'}
Kortnavn: test_shortname
Definisjon: {'nb': 'ny definisjon2', 'nn': 'ny definisjon2', 'en': 'new definition2'}
Eier: {'team': 'dapla-felles', 'groups': ['dapla-felles-developers', 'play-enhjoern-a-developers']}



### Filtrere på dato

Nyttig hvis man ønsker tilgang til en spesifikk gyldighetsperiode.


In [ ]:

desired_validity = date(2030,2,24)
filtered_variable_definitions = Vardef.list_variable_definitions(desired_validity)
print(f"Valid at {desired_validity}:\n")
for variable in filtered_variable_definitions:
    print(f"Id: {variable.id}\nName: {variable.name}\nShort name: {variable.short_name}\nDefinition: {variable.definition}\nOwner: {variable.owner}\nValid: {variable.valid_from} to {variable.valid_until}\n")

### Skrive egne filteroperasjoner

Vardef støtter ikke annen filtrering enn dato, men det er enkelt å filtrere på ønsket attributt og tilpasse resultatet etter behov.

Under følger noen eksempler der man filtrere på eierteam og status.




In [20]:
print("\nFiltrer etter eierteam: ")
my_team_variables = [variable for variable in variable_definitions if variable.owner["team"] == "dapla-felles"]
print(my_team_variables)

print("\nFiltrer etter status `DRAFT`: ")
draft_variables = [variable for variable in variable_definitions if variable.variable_status == VariableStatus.DRAFT]
print(draft_variables)

print("\nFiltrer etter status `PUBLISHED INTERNAL`: ")
published_intern_variables = [variable for variable in variable_definitions if variable.variable_status == VariableStatus.PUBLISHED_INTERNAL]
print(published_intern_variables)



Filtrer etter eierteam: 
[VariableDefinition(id='QlTc_6H5', patch_id=4, name={'nb': 'test navn', 'nn': 'test namn', 'en': 'test name'}, short_name='test_shortname', definition={'nb': 'ny definisjon2', 'nn': 'ny definisjon2', 'en': 'new definition2'}, classification_reference='91', unit_types=['01'], subject_fields=['al'], contains_special_categories_of_personal_data=False, variable_status=<VariableStatus.PUBLISHED_INTERNAL: 'PUBLISHED_INTERNAL'>, measurement_type=None, valid_from=datetime.date(2040, 4, 5), valid_until=None, external_reference_uri=None, comment=None, related_variable_definition_uris=None, owner={'team': 'dapla-felles', 'groups': ['dapla-felles-developers', 'play-enhjoern-a-developers']}, contact={'title': {'nb': 'Seksjon for befolkningsstatistikk', 'nn': 'Seksjon for befolkningsstatistikk', 'en': 'Division for population statistics'}, 'email': 'sibby@ssb.no'}, created_at=datetime.datetime(2024, 12, 20, 15, 20, 57, 647000), created_by=None, last_updated_at=datetime.date

#### Hent en variabeldefinisjon

Hent en ved å sende inn `id`. 


In [21]:
print(Vardef.get_variable_definition(my_draft.id))

{
  "id": "QlTc_6H5",
  "patch_id": 4,
  "name": {
    "nb": "test navn",
    "nn": "test namn",
    "en": "test name"
  },
  "short_name": "test_shortname",
  "definition": {
    "nb": "ny definisjon2",
    "nn": "ny definisjon2",
    "en": "new definition2"
  },
  "classification_reference": "91",
  "unit_types": [
    "01"
  ],
  "subject_fields": [
    "al"
  ],
  "contains_special_categories_of_personal_data": false,
  "variable_status": "PUBLISHED_INTERNAL",
  "measurement_type": null,
  "valid_from": "2040-04-05",
  "valid_until": null,
  "external_reference_uri": null,
  "comment": null,
  "related_variable_definition_uris": null,
  "owner": {
    "team": "dapla-felles",
    "groups": [
      "dapla-felles-developers",
      "play-enhjoern-a-developers"
    ]
  },
  "contact": {
    "title": {
      "nb": "Seksjon for befolkningsstatistikk",
      "nn": "Seksjon for befolkningsstatistikk",
      "en": "Division for population statistics"
    },
    "email": "sibby@ssb.no"
  },


#### Hente en variabeldefinisjon ved hjelp av kortnavn

Hvis man mister oversikten over `id` er dette et eksempel på hvordan man kan hente ut `id` ved å bruke kortnavn som også er unikt.

In [ ]:

variable_by_short_name = next(variable for variable in variable_definitions if variable.short_name == "test_shortname")
print(variable_by_short_name.id)

## Alternativer til `Model` input

Hvis man ønsker det er det også mulig å sende inn `dictionary`.

In [22]:
NEW_DRAFT = {
    "name": {"en": "Country Background", "nb": "Landbakgrunn", "nn": "Landbakgrunn"},
    "short_name": "new_short_name1",
    "definition": {
        "en": "Country background is the person's own, the mother's or possibly the father's country of birth. Persons without an immigrant background always have Norway as country background. In cases where the parents have different countries of birth the mother's country of birth is chosen. If neither the person nor the parents are born abroad, country background is chosen from the first person born abroad in the order mother's mother, mother's father, father's mother, father's father.",
        "nb": "For personer født i utlandet, er dette (med noen få unntak) eget fødeland. For personer født i Norge er det foreldrenes fødeland. I de tilfeller der foreldrene har ulikt fødeland, er det morens fødeland som blir valgt. Hvis ikke personen selv eller noen av foreldrene er utenlandsfødt, hentes landbakgrunn fra de første utenlandsfødte en treffer på i rekkefølgen mormor, morfar, farmor eller farfar.",
        "nn": "For personar fødd i utlandet, er dette (med nokre få unntak) eige fødeland. For personar fødd i Noreg er det fødelandet til foreldra. I dei tilfella der foreldra har ulikt fødeland, er det fødelandet til mora som blir valt. Viss ikkje personen sjølv eller nokon av foreldra er utenlandsfødt, blir henta landsbakgrunn frå dei første utenlandsfødte ein treffar på i rekkjefølgja mormor, morfar, farmor eller farfar.",
    },
    "classification_reference": "91",
    "unit_types": ["01", "02"],
    "subject_fields": ["he04"],
    "contains_special_categories_of_personal_data": True,
    "measurement_type": None,
    "valid_from": "2003-01-01",
    "external_reference_uri": "https://www.ssb.no/a/metadata/conceptvariable/vardok/1919/nb",
    "comment": {
        "nb": "Fra og med 1.1.2003 ble definisjon endret til også å trekke inn besteforeldrenes fødeland.",
        "nn": "Fra og med 1.1.2003 ble definisjon endret til også å trekke inn besteforeldrenes fødeland.",
        "en": "As of 1 January 2003, the definition was changed to also include the grandparents' country of birth.",
    },
    "related_variable_definition_uris": ["https://example.com/"],
    "contact": {
        "title": {
            "en": "Division for population statistics",
            "nb": "Seksjon for befolkningsstatistikk",
            "nn": "Seksjon for befolkningsstatistikk",
        },
        "email": "s320@ssb.no",
    },
}

